**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten, Dropout

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e): #Used in the question 10
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The objective of the function act is to return the next action o do, but there is a trade-off to do between the exploration of the environment - for which is used the random walk and the exploitation of a policy. <br>
The objective of the parameter $\epsilon$ is to define the probability to do  a random action. If $\epsilon$ is too big,  the randm walk influence will be too important and the test of a policy will be too long. If $\epsilon$ is too low the environment will not be well known and the learned policy may not be optimal.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 #typo

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 #typo
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=50 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.



1.   Use of the position array:
 We can decompose the elements of the position array in 3 parts:<br>
  * The elements of the grid which are not accessible. It corresponds to the contour of the array with a width of 2 (i.e the two first and the two last lines, and the two first and the two last columns) - this contour corresponds to a wall where the agent cannot go but rebounces, but posing its existence this way simplifies the code with regard to the knowledge of the surrounding states part.
  * The elements of the grid with a value 1. There is only one element with this value, and it corresponds to the current position of the agent
  * The other states, which corresponds to the states theoretically accessible but not currently occupied, and to which are associated a value of 0.
  
 And so, updating the position of the agent corresponds to change the aspect of the position array, by putting to zero the value associated to the old position of the agent and to 1 the the value associated to the new position of the agent - the coordinates of this position being saved in (x, y).

2.  Use of the board array:
The board array has also the contour with a width of 2 representing some inaccessible states. But the value taken on the coordinate (x, y) is the value of the reward function on the corresponding state - this one is randomly initialized and when the agent goes on a state, it "consumes" the reward function there, ad so the reward function in this state will return zero.

The advantage of the use of arrays is that it is an understandable and simple way to store global information about the current characteristics  of our environment.




## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # We randomly select an action, no matter the state
        # In this case, the strategy is always the same no matter the value of epsilon
        action = np.random.randint(0, self.n_action, size=1)[0]
        return action

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
    
        # At each epoch, we restart to a fresh game and get the initial state
      state = env.reset()
      # This assumes that the games will end
      game_over = False

      win = 0
      lose = 0

      while not game_over:
          # The agent performs an action
          action = agent.act(state, train = False)

          # Apply an action to the environment, get the next state, the reward
          # and if the games end
          prev_state = state
          state, reward, game_over = env.act(action)

          # Update the counters
          if reward > 0:
              win = win + reward
          if reward < 0:
              lose = lose -reward         
        
      # Save as a mp4
      env.draw(prefix+str(e))

      # Update stats
      score = score + win-lose

      print("Win/lose count {}/{}. Average score ({})"
                .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 6.5/9.0. Average score (-2.5)
Win/lose count 5.5/10.0. Average score (-3.5)
Win/lose count 12.5/12.0. Average score (-2.1666666666666665)
Win/lose count 13.5/16.0. Average score (-2.25)
Win/lose count 8.0/4.0. Average score (-1.0)
Win/lose count 8.5/9.0. Average score (-0.9166666666666666)
Win/lose count 10.0/11.0. Average score (-0.9285714285714286)
Win/lose count 7.5/6.0. Average score (-0.625)
Win/lose count 5.5/8.0. Average score (-0.8333333333333334)
Win/lose count 5.5/11.0. Average score (-1.3)
Win/lose count 11.0/12.0. Average score (-1.2727272727272727)
Win/lose count 11.5/12.0. Average score (-1.2083333333333333)
Win/lose count 14.5/12.0. Average score (-0.9230769230769231)
Win/lose count 7.0/19.0. Average score (-1.7142857142857142)
Win/lose count 10.0/11.0. Average score (-1.6666666666666667)
Win/lose count 11.0/21.0. Average score (-2.1875)
Win/lose count 7.5/13.0. Average score (-2.3823529411764706)
Win/lose count 11.0/17.0. Average score (-2.583333333333333

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




I]
\begin{equation*}Q^\pi(s,a)=E_{p^\pi}[\sum_{t=0}^{\infty} \gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]  \\ = r(s ,a) + E_{p^\pi}[\sum_{t=1}^{\infty} \gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\ = r(s ,a) + \gamma E_{p^\pi}[\sum_{t=0}^{\infty} \gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a] \\ = r(s, a) + \gamma\sum_{s', a'} E_{p^\pi}[\sum_{t=0}^{\infty} \gamma^{t}r(\hat{s}_{t},\hat{a}_{t})|\hat{s}_{0}=s' ,\hat{a}_{0}=a'] P_\pi(s', a'| s_0 = s, a_0 = a) \\  =  r(s, a) + \gamma\sum_{s', a'} Q^\pi (s', a')P_\pi(s', a'| s_0 = s, a_0= a) \\ = E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] ( \text{ as r(s,a) is constant})\end{equation*}


II]
\begin{equation*}
Q^{\star}(s,a)= \max\limits_\pi Q^\pi (s,a) = \max\limits_\pi E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\ = r(s,a) +\gamma \max\limits_\pi E_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')]  \\ \end{equation*}
Let's show that $\max\limits_\pi E_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')]$ = $E_{(s',a')\sim p(.|s,a)}[\max\limits_\pi Q^{\pi}(s',a')]:$ <br>
\begin{equation*}
\max\limits_\pi E_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')] = \max\limits_\pi \sum_{s', a'} p(s', a'| s, a) Q^{\pi}(s',a') =  \max\limits_\pi \sum_{s', a'} p(s'| s, a) \pi(a'|s') Q^{\pi}(s',a')
\end{equation*}
But, $\forall \pi, \max\limits_\pi \sum_{s'} \sum_{a'} p(s'| s, a) \pi(a'|s') Q^{\pi}(s',a') \leq \sum_{s'} \max\limits_{a'} p(s'| s, a) Q^{\star}(s',a') = \sum_{s'} \sum_{a'} p(s'| s, a) \pi^\star(a'|s') Q^{\star}(s',a')$, as $0 \leq \pi(a'|s') \leq 1$, $\sum_{a'} \pi(a'|s') = 1$ and by the definition of $Q^{\star}$.<br>
Moreover, we have the other inequality by considering that the evaluation on the strategy $\pi^\star$ is lower than the $\max\limits_\pi$.

So, we have $Q^{\star}(s,a)= r(s, a) + E_{(s')\sim p(.|s,a)}[\max\limits_{a'} Q^{\star}(s',a')]$


III]


Let $Q(s, a; \theta)$ be the approximation of the optimal Q-value function parametrized by $\theta$. If we use the classic quadratic loss function, we can say that
$\mathcal{L}(\theta) = E_{s' \sim \pi^*(.|s,a)}[\Vert Q^\star (s, a)-Q(s,a,\theta)\Vert^{2}].$ But how to estimate $Q^\star$ ?
For that, we will use the formula defined on the point 2. with $Q(s, a, \theta)$, as it will be equal to 0 if we have the good $Q^\star$:
$ E_{s' \sim \pi^*(.|s,a)}[\Vert r+\gamma \max_{a'}Q(s',a',\theta) -Q(s,a,\theta)\Vert^{2}]$





***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        (self.memory).append(m)
        self.memory = self.memory[-1*self.max_memory:]

    def random_access(self):
        l = np.random.randint(0, len(self.memory))
        return self.memory[l]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)

In [12]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s, 0)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            b_s_, b_n_s_, b_a_, b_r_, b_game_over_ = self.memory.random_access()
            input_states[i] = b_s_
            target_q[i] = self.model.predict(np.expand_dims(b_s_, 0))
            
            if b_game_over_:
                target_q[i, b_a_] = b_r_
            else:
                target_q[i, b_a_] = b_r_ + self.discount * np.max(self.model.predict(np.expand_dims(b_n_s_, 0)))
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1, **kwargs):
        self.lr = lr
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        #model.add(Dense(30, input_shape=(5, 5, self.n_state), activation='relu'))
        #model.add(Dense(30, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Dense(50, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [13]:
agent = DQN_FC(size, lr= 0.1, epsilon = 0.1, memory_size=1000, batch_size = 64)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/050 | Loss 0.0165 | Win/lose count 3.0/0 (3.0)
Epoch 001/050 | Loss 0.0258 | Win/lose count 3.5/3.0 (0.5)
Epoch 002/050 | Loss 0.0195 | Win/lose count 1.0/1.0 (0.0)
Epoch 003/050 | Loss 0.1031 | Win/lose count 6.0/6.0 (0.0)
Epoch 004/050 | Loss 0.0084 | Win/lose count 1.5/6.0 (-4.5)
Epoch 005/050 | Loss 0.0025 | Win/lose count 2.0/1.0 (1.0)
Epoch 006/050 | Loss 0.0044 | Win/lose count 4.0/2.0 (2.0)
Epoch 007/050 | Loss 0.0068 | Win/lose count 2.5/6.0 (-3.5)
Epoch 008/050 | Loss 0.0033 | Win/lose count 4.0/4.0 (0.0)
Epoch 009/050 | Loss 0.0050 | Win/lose count 3.5/5.0 (-1.5)
Epoch 010/050 | Loss 0.0057 | Win/lose count 4.0/0 (4.0)
Epoch 011/050 | Loss 0.0020 | Win/lose count 7.0/6.0 (1.0)
Epoch 012/050 | Loss 0.0290 | Win/lose count 5.5/6.0 (-0.5)
Epoch 013/050 | Loss 0.0026 | Win/lose count 6.5/2.0 (4.5)
Epoch 014/050 | Loss 0.0044 | Win/lose count 3.0/1.0 (2.0)
Epoch 015/050 | Loss 0.0049 | Win/lose count 2.0/0 (2.0)
Epoch 016/050 | Loss 0.0284 | Win/lose count 4.0/2.0 (2.0)

In [15]:
print('Test of the FC')
agent_fc = DQN_FC(size, lr=0.1, epsilon = 0.1, memory_size=1000, batch_size = 64)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the FC
Win/lose count 2.0/0. Average score (2.0)
Win/lose count 2.0/0. Average score (2.0)
Win/lose count 0/0. Average score (1.3333333333333333)
Win/lose count 6.5/0. Average score (2.625)
Win/lose count 3.0/0. Average score (2.7)
Win/lose count 1.0/0. Average score (2.4166666666666665)
Win/lose count 2.0/0. Average score (2.357142857142857)
Win/lose count 0.5/0. Average score (2.125)
Win/lose count 5.0/1.0. Average score (2.3333333333333335)
Win/lose count 8.0/0. Average score (2.9)
Win/lose count 0.5/0. Average score (2.6818181818181817)
Win/lose count 2.0/0. Average score (2.625)
Win/lose count 5.5/0. Average score (2.8461538461538463)
Win/lose count 7.0/0. Average score (3.142857142857143)
Win/lose count 2.5/0. Average score (3.1)
Win/lose count 4.0/0. Average score (3.15625)
Win/lose count 4.0/0. Average score (3.2058823529411766)
Win/lose count 5.0/0. Average score (3.3055555555555554)
Win/lose count 3.0/0. Average score (3.289473684210526)
Win/lose count 6.0/0. Average 

After several tests, we have obtain the following results: <br>
* The structure composed of only one fully connected layer seems to work better than the structures with several fully connected layers (at least it works better than  the network with 2 or 3 layers)
* The number of neurons in the layer seems to _____
* An increase of the batch size increases the performance.
* A too important memory size seems to decrease the performance of the DQN.
* The use of strategies like batch-normalization are not adapted to our case.


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [31]:
class DQN_CNN(DQN):
    def __init__(self, lr=0.1, *args, **kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(50, 3, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Conv2D(30, 3, activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))

        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [32]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(0.1, size, epsilon = 0.1, memory_size=1000, batch_size = 64)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0096 | Win/lose count 4.0/6.0 (-2.0)
Epoch 001/050 | Loss 0.0031 | Win/lose count 3.5/2.0 (1.5)
Epoch 002/050 | Loss 0.0073 | Win/lose count 1.5/2.0 (-0.5)
Epoch 003/050 | Loss 0.0114 | Win/lose count 1.0/5.0 (-4.0)
Epoch 004/050 | Loss 0.0015 | Win/lose count 3.5/2.0 (1.5)
Epoch 005/050 | Loss 0.0269 | Win/lose count 9.0/2.0 (7.0)
Epoch 006/050 | Loss 0.0022 | Win/lose count 6.5/4.0 (2.5)
Epoch 007/050 | Loss 0.0314 | Win/lose count 8.0/4.0 (4.0)
Epoch 008/050 | Loss 0.0022 | Win/lose count 3.5/3.0 (0.5)
Epoch 009/050 | Loss 0.0293 | Win/lose count 11.5/5.0 (6.5)
Epoch 010/050 | Loss 0.0309 | Win/lose count 12.0/5.0 (7.0)
Epoch 011/050 | Loss 0.0271 | Win/lose count 5.5/5.0 (0.5)
Epoch 012/050 | Loss 0.0022 | Win/lose count 11.5/1.0 (10.5)
Epoch 013/050 | Loss 0.0034 | Win/lose count 5.0/1.0 (4.0)
Epoch 014/050 | Loss 0.0023 | Win/lose count 13.0/4.0 (9.0)
Epoch 015/050 | Loss 0.0036 | Win/lose count 12.5/1.0 (11.5)
Epoch 016/050 | Loss 0.0296 | Win/lose count 8

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [42]:
env = Environment(grid_size=size, max_time=T,temperature=0.8)
agent_cnn = DQN_CNN(0.1, size, epsilon = 0.1, memory_size=1000, batch_size = 64)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=0.1, epsilon = 0.1, memory_size=1000, batch_size = 64)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 10.0/1.0. Average score (9.0)
Win/lose count 17.0/0. Average score (13.0)
Win/lose count 24.5/1.0. Average score (16.5)
Win/lose count 9.5/0. Average score (14.75)
Win/lose count 31.0/0. Average score (18.0)
Win/lose count 25.0/1.0. Average score (19.0)
Win/lose count 34.5/0. Average score (21.214285714285715)
Win/lose count 25.0/0. Average score (21.6875)
Win/lose count 48.5/0. Average score (24.666666666666668)
Win/lose count 22.0/0. Average score (24.4)
Win/lose count 26.0/0. Average score (24.545454545454547)
Win/lose count 20.5/0. Average score (24.208333333333332)
Win/lose count 47.5/3.0. Average score (25.76923076923077)
Win/lose count 19.0/0. Average score (25.285714285714285)
Win/lose count 6.0/0. Average score (24.0)
Win/lose count 23.0/0. Average score (23.9375)
Win/lose count 35.0/2.0. Average score (24.470588235294116)
Win/lose count 23.5/0. Average score (24.416666666666668)
Win/lose count 3.0/0. Average score (23.289473684210527)
Win/lose c

In [41]:
HTML(display_videos('cnn_test10.mp4'))

Training made with a temperature of 0.3 <br>

| Temperature      |     Score CNN   |   Score FC |
| ------------- |: -------------: | ---------: |
| 0.1     |        1.96      |     1.66|
| 0.2   |        3.8      |     3.56|
| 0.3      |        6.13       |      5.06 |
| 0.4   |           9.58         | 8.78
| 0.5   |        10.85        |      9.44 |
| 0.6      |        14.86       |      16.1 |
| 0.7      |        21.04       |      21.21 |
| 0.8      |        27.19       |      33.4 |
| 0.9      |        36.75       |      44.25 |


In [19]:
HTML(display_videos('fc_test10.mp4'))

We can remark the following problems:
* First, the fact that the $\epsilon$ is a constant makes that, at the beginning, it will have a tendancy to choose the "learnt" strategy even though it hasn't learnt anything and that, at the end, it will tend to explore whereas it  is no more a necessity
* Moreover, we can remark on the videos that the agent may stay on a zone that it has already explored and so where there is no more reward - for example, it can altern between two neighbors states already explored. We can note that it happens especially when there is no positive rewards around the agent.
* Finally, we can remark that the bigger the temperature is, the bigger is the obtained score for both algorithms. But, if for low temperatures the CNN network has better performance, for high temperatures it is the fully-connected network which works better - for high temperatures, the CNN is more likely to have the behaviour described above.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [43]:
def train_explore(agent, env, epoch, epsilon_decay = 0.95, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        agent.set_epsilon(agent.epsilon * epsilon_decay)
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train = True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 #typo

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward += self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 #typo
        self.board[self.x,self.y] = 0
        self.t = 0

        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    
## use those samples of code:
#In train explore:
#state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
#reward = 0
#if train:
#    reward = -self.malus_position[self.x, self.y]
#self.malus_position[self.x, self.y] = 0.1

#reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
#state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                self.board.reshape(self.grid_size, self.grid_size,1),
#                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [44]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(0.1, size, epsilon = 0.5, memory_size=1000, batch_size = 64,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0071 | Win/lose count 5.0/29.00000000000014 (-24.00000000000014)
Epoch 001/050 | Loss 0.0123 | Win/lose count 14.5/20.10000000000004 (-5.6000000000000405)
Epoch 002/050 | Loss 0.0090 | Win/lose count 10.0/19.200000000000014 (-9.200000000000014)
Epoch 003/050 | Loss 0.0180 | Win/lose count 5.0/21.600000000000012 (-16.600000000000012)
Epoch 004/050 | Loss 0.0097 | Win/lose count 15.5/19.400000000000023 (-3.9000000000000234)
Epoch 005/050 | Loss 0.0066 | Win/lose count 11.0/21.700000000000014 (-10.700000000000014)
Epoch 006/050 | Loss 0.0058 | Win/lose count 13.0/19.90000000000003 (-6.9000000000000306)
Epoch 007/050 | Loss 0.0095 | Win/lose count 18.0/16.399999999999974 (1.6000000000000263)
Epoch 008/050 | Loss 0.0408 | Win/lose count 16.0/20.50000000000001 (-4.500000000000011)
Epoch 009/050 | Loss 0.0068 | Win/lose count 5.0/21.100000000000026 (-16.100000000000026)
Epoch 010/050 | Loss 0.0305 | Win/lose count 19.5/18.299999999999983 (1.200000000000017)
Epoch 011/05

In [46]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 23.5/0. Average score (23.5)
Win/lose count 23.0/0. Average score (23.25)
Win/lose count 23.0/1.0. Average score (22.833333333333332)
Win/lose count 20.0/1.0. Average score (21.875)
Win/lose count 16.5/1.0. Average score (20.6)
Win/lose count 13.5/1.0. Average score (19.25)
Win/lose count 6.0/2.0. Average score (17.071428571428573)
Win/lose count 21.5/1.0. Average score (17.5)
Win/lose count 17.0/2.0. Average score (17.22222222222222)
Win/lose count 29.0/1.0. Average score (18.3)
Win/lose count 29.5/3.0. Average score (19.045454545454547)
Win/lose count 19.5/1.0. Average score (19.0)
Win/lose count 24.5/1.0. Average score (19.346153846153847)
Win/lose count 4.5/0. Average score (18.285714285714285)
Win/lose count 20.0/3.0. Average score (18.2)
Win/lose count 22.0/0. Average score (18.4375)
Win/lose count 19.0/1.0. Average score (18.41176470588235)
Win/lose count 13.0/3.0. Average score (17.944444444444443)
Win/lose count 20.0/1.0. Average score (18.0)
Win/lose count 13.5

We can see that, for a same temperature, this algorithm outperforms the previous version (by a factor close to 3).

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***